<a href="https://colab.research.google.com/github/gibsonx/CE888/blob/master/skeleton_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.11.0 True


In [2]:
!pip show pyyaml
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' &> /dev/null

Name: PyYAML
Version: 6.0
Summary: YAML parser and emitter for Python
Home-page: https://pyyaml.org/
Author: Kirill Simonov
Author-email: xi@resolvent.net
License: MIT
Location: /opt/conda/lib/python3.7/site-packages
Requires: 
Required-by: accelerate, albumentations, autocfg, bokeh, cliff, cookiecutter, dask, distributed, easyocr, essentia, fastai, flax, gluoncv, huggingface-hub, jupytext, kubernetes, nnabla, optuna, pandas-profiling, papermill, pyaml, pytorch-lightning, quilt3, ray, rvlib, transformers, vaex-core, wandb, yacs


In [3]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [4]:
from detectron2.data.datasets import register_coco_instances


register_coco_instances("person_keypoints_train", {}, "/kaggle/input/coco-2017-dataset/coco2017/annotations/person_keypoints_train2017.json", "/kaggle/input/coco-2017-dataset/coco2017/train2017")

register_coco_instances("person_keypoints_val", {}, "/kaggle/input/coco-2017-dataset/coco2017/annotations/person_keypoints_val2017.json", "/kaggle/input/coco-2017-dataset/coco2017/val2017")

In [5]:
from detectron2.data import MetadataCatalog
MetadataCatalog.get("person_keypoints_train").thing_classes = ["person"]

In [6]:
keypoint_names = ["nose","left_eye","right_eye","left_ear","right_ear","left_shoulder","right_shoulder","left_elbow","right_elbow","left_wrist","right_wrist","left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"]
keypoint_flip_map = [('left_eye', 'right_eye'),]

In [7]:
len(keypoint_names)

17

In [8]:
MetadataCatalog.get("person_keypoints_train").thing_dataset_id_to_contiguous_id = {1:0}
MetadataCatalog.get("person_keypoints_train").keypoint_names = keypoint_names
MetadataCatalog.get("person_keypoints_train").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("person_keypoints_train").evaluator_type="coco"

In [9]:
dataset_dicts = DatasetCatalog.get("person_keypoints_train")
dataset_dicts[0]

[02/16 14:46:02 d2.data.datasets.coco]: Loading /kaggle/input/coco-2017-dataset/coco2017/annotations/person_keypoints_train2017.json takes 11.42 seconds.
[02/16 14:46:03 d2.data.datasets.coco]: Loaded 118287 images in COCO format from /kaggle/input/coco-2017-dataset/coco2017/annotations/person_keypoints_train2017.json


{'file_name': '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000000009.jpg',
 'height': 480,
 'width': 640,
 'image_id': 9,
 'annotations': []}

#Begin Training

In [10]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
# cfg.MODEL.DEVICE = "cpu"


cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("person_keypoints_train",)


#cfg.DATASETS.TEST = ("hand_test",)  #Dataset 'hand_test' is empty in my case
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 2000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   #128   # faster, and good enough for this toy dataset (default: 512)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # person
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 17
# cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((17, 1), dtype=float).tolist()
# del cfg.TEST.KEYPOINT_OKS_SIGMAS


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)    #CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/16 14:46:15 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_a6e10b.pkl: 237MB [00:02, 90.9MB/s]                           


[02/16 14:46:43 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.7/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/opt/conda/lib/python3.7/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be requ

[02/16 14:46:57 d2.utils.events]:  eta: 0:10:44  iter: 19  total_loss: 6.056  loss_cls: 0.1094  loss_box_reg: 0.09689  loss_keypoint: 5.71  loss_rpn_cls: 0.006094  loss_rpn_loc: 0.01163  time: 0.3297  last_time: 0.3087  data_time: 0.0228  last_data_time: 0.0008   lr: 4.9953e-06  max_mem: 2780M
[02/16 14:47:04 d2.utils.events]:  eta: 0:10:49  iter: 39  total_loss: 4.286  loss_cls: 0.09841  loss_box_reg: 0.1275  loss_keypoint: 3.919  loss_rpn_cls: 0.007297  loss_rpn_loc: 0.01719  time: 0.3381  last_time: 0.3073  data_time: 0.0083  last_data_time: 0.0010   lr: 9.9902e-06  max_mem: 2780M
[02/16 14:47:10 d2.utils.events]:  eta: 0:10:39  iter: 59  total_loss: 5.294  loss_cls: 0.1086  loss_box_reg: 0.1574  loss_keypoint: 4.972  loss_rpn_cls: 0.003821  loss_rpn_loc: 0.02701  time: 0.3343  last_time: 0.3440  data_time: 0.0076  last_data_time: 0.0041   lr: 1.4985e-05  max_mem: 2780M
[02/16 14:47:17 d2.utils.events]:  eta: 0:10:36  iter: 79  total_loss: 4.885  loss_cls: 0.09793  loss_box_reg: 0.1